In [1]:
import pyomo.environ as pmo
import numpy as np

class RedundancyChecker:
    
    def __init__(self, A, b, tol=1e-7):
        self.A = A
        self.b = b
        self.tol = tol
        
        # get no of var and constraints
        self.x_card = np.shape(A)[1]
        self.c_card = np.shape(A)[0]
        
        # transform A from matrix to dict
        A_init = {}
        for i in range(self.c_card):
            for j in range(self.x_card):
                A_init[(i+1, j+1)] = A[i, j]
        
        # transform b from vector to dict
        b_init = {}
        for i in range(self.c_card):
            b_init[i+1] = b[i]
            
        # define pyomo model
        self.model = pmo.ConcreteModel()
        self.model.n = pmo.RangeSet(1, self.x_card)
        self.model.c = pmo.RangeSet(1, self.c_card)
        self.model.A = pmo.Param(self.model.c, self.model.n, initialize=A_init)
        self.model.b = pmo.Param(self.model.c, initialize=b_init)
        self.model.x = pmo.Var(self.model.n)
        self.model.dual = pmo.Suffix(direction=pmo.Suffix.IMPORT)
        self.model.constraints = pmo.ConstraintList()
        for c in self.model.c:
            self.model.constraints.add(
                sum(self.model.A[c, i] * self.model.x[i] for i in self.model.n) <= self.model.b[c]
            )
        
        # define solver
        self.solverpath = 'C:\\w64\\glpsol'
        self.solver = pmo.SolverFactory('glpk', executable=self.solverpath)
#         self.solver = pmo.SolverFactory('cplex')
    
        # define empty output entities
        self.redundancy = None
        self.reduced_A = None
        self.reduced_b = None
        
    def check(self):
        # for each constraint, delete any old obj, set new obj as Ax of chosen constraint
        # and maximise it.
        # Deactivate the chosen constraint itself.
        # Then check if b-Ax to see if positive (constraint is loose).
        # If so, mark as redundant.
        self.redundancy = np.zeros([self.c_card])
        self.slack = np.zeros([self.c_card])
        for c in self.model.c:
            try:
                self.model.del_component(self.model.obj)
            except:
                pass
            self.model.obj = pmo.Objective(
                expr=-sum(self.model.A[c, i] * self.model.x[i] for i in self.model.n)
            )
            #self.model.constraints[c].deactivate()
            self.solver.solve(self.model, tee=False)
            #self.model.constraints[c].activate()
            
            self.slack[c-1] = self.model.b[c] + pmo.value(self.model.obj)

            if self.model.b[c] + pmo.value(self.model.obj) > self.tol:
                self.redundancy[c-1] = 1
    
        print('slack=')
        print(self.slack)
        self.reduced_A = self.A[self.redundancy == 0]
        self.reduced_b = self.b[self.redundancy == 0]

In [2]:
# check the class works
A = np.array(
    [[-0.13793103448275856, 4.206896551724137],
     [1.0, 0.0],
     [0.0, 1.0]]
)

b = np.array([896.5517241379312, 6000.0, 500.0])
mymodel = RedundancyChecker(A, b)
mymodel.check()
print(mymodel.reduced_A)
print(mymodel.reduced_b)

slack=
[-2.04636308e-12  0.00000000e+00  9.01639344e+01]
[[-0.13793103  4.20689655]
 [ 1.          0.        ]]
[ 896.55172414 6000.        ]
